In [51]:
import pandas as pd
import os

In [52]:
path_folder = "/Users/matteolemesre/Desktop/Data LOSC/csv/csv24_25/"
path_start_players  = os.path.join(path_folder, "clean/data_players.csv")
path_start_goals    = os.path.join(path_folder, "clean/data_goals.csv")
path_end = os.path.join(path_folder, "clean/data_teams.csv")

In [53]:
players_df = pd.read_csv(os.path.join(path_folder, path_start_players))
goals_df = pd.read_csv(os.path.join(path_folder, path_start_goals))

In [54]:
team_stats_players = players_df.groupby(['Game Week', 'League', 'Team']).sum(numeric_only=True).reset_index()
team_stats_goals = goals_df.groupby(['Game Week', 'League', 'Team']).sum(numeric_only=True).reset_index()

team_stats = pd.merge(
    team_stats_players,
    team_stats_goals,
    on=['Game Week', 'League', 'Team'],
    how='outer',
    suffixes=('', '_goal')
)

In [55]:
adversaires = []

championnats = team_stats['League'].unique()

for champ in championnats:
    path = os.path.join(path_folder, f"leagues_games/{champ}_games.csv")
    if not os.path.exists(path):
        print(f"Fichier non trouvé : {path}")
        continue

    match_df = pd.read_csv(path)

    match_df = match_df.rename(columns={
        'Home Team': 'Home',
        'Away Team': 'Away'
    })

    match_df['League'] = champ

    home_df = match_df[['Game Week', 'League', 'Home', 'Away']].copy()
    home_df = home_df.rename(columns={'Away': 'Opponent'})

    away_df = match_df[['Game Week', 'League', 'Home', 'Away']].copy()
    away_df = away_df.rename(columns={'Home': 'Opponent'})

    match_long = pd.concat([home_df, away_df])
    adversaires.append(match_long)

adversaires_df = pd.concat(adversaires)



In [56]:
stats_with_opp = pd.merge(
    team_stats,
    adversaires_df,
    on=['Game Week', 'League'],
    how='left'
)

stats_with_opp.drop(columns=['Minutes', 'Minutes_goal', 'Efficacité des arrêts', 'Pourcentage d arrêts'], inplace=True, errors='ignore')
stats_with_opp.to_csv(path_end, index=False)